# Analisa Data Harga Rumah di Boston  
Setiap data dalam basis data ini menggambarkan sebuah daerah pinggiran atau kota di Boston. Data ini diambil dari wilayah Boston Standard Metropolitan Statistical Area (SMSA) pada tahun 1970.  
**Sumber data :** (https://www.kaggle.com/datasets/vikrishnan/boston-house-prices/data)

| Nama Kolom  | Deskripsi                                                         |
| ----------- | ---------------------------------------------------------------------------------------- |
| **CRIM**    | Tingkat kejahatan per kapita di setiap kota                                              |
| **ZN**      | Proporsi lahan perumahan yang diperuntukkan untuk kavling lebih dari 25.000 kaki persegi |
| **INDUS**   | Proporsi luas area bisnis non-ritel di setiap kota                                       |
| **CHAS**    | Variabel dummy Sungai Charles (1 jika wilayah berbatasan dengan sungai, 0 jika tidak)    |
| **NOX**     | Konsentrasi nitrogen oksida (dalam satuan per 10 juta)                                   |
| **RM**      | Rata-rata jumlah kamar per rumah tinggal                                                 |
| **AGE**     | Proporsi rumah yang dihuni pemilik dan dibangun sebelum tahun 1940                       |
| **DIS**     | Jarak tertimbang ke lima pusat pekerjaan di Boston                                       |
| **RAD**     | Indeks aksesibilitas ke jalan raya radial                                                |
| **TAX**     | Tarif pajak properti penuh per \$10.000 nilai properti                                   |
| **PTRATIO** | Rasio murid-guru di tiap kota                                                            |
| **B**       | 1000 × (Bk − 0,63)², di mana Bk adalah proporsi penduduk kulit hitam di kota tersebut    |
| **LSTAT**   | Persentase penduduk dengan status sosial ekonomi rendah                                  |


### Import Data

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, Normalizer

In [ ]:
data =pd.read_csv('data/housing.csv')
data.head()

### Preprocessing

In [ ]:
data =pd.read_csv('data/housing.csv', sep='\s+', header=None)
data.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']

In [ ]:
data.head()

In [ ]:
#cek ukuran data
data.shape

In [ ]:
#cek tipe data yang digunakan
data.dtypes

In [ ]:
#identifikasi isi dataset yang uniq
data.nunique()

In [ ]:
#cek missing value
data.isnull().sum()

In [ ]:
#cek baris yang memiliki missing value
data[data.isnull().any(axis=1)]

In [ ]:
#melihat statistik deskriptif dari data
data.describe()

In [ ]:
data.to_excel('data/housing_cleaned.xlsx', index=False)

### Research Question
1. Faktor apa saja yang paling berpengaruh terhadap harga rumah di Boston?
2. Apakah tingkat kejahatan (CRIM) berpengaruh negatif terhadap harga rumah?
3. Apakah rasio murid-guru (PTRATIO) berkorelasi dengan harga rumah?
4. Apakah kedekatan dengan Sungai Charles (CHAS) berdampak signifikan terhadap harga rumah?
5. Apakah rumah di area dengan pajak tinggi (TAX) memiliki harga jual lebih rendah?
6. Bagaimana pengaruh status sosial ekonomi (LSTAT) terhadap harga rumah?

#### 1. Faktor yang paling berpengaruh terhadap harga rumah

In [ ]:
corr = data.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')

**hasil** :
Yang paling berpengaruh terhadap harga rumah adalah **RM, ZN, B, DIS**  
Jadi harga rumah di Boston paling dipengaruhi oleh:
- Rata - rata jumlah kamar per rumah tinggal
- Proposi lahan perumahan yang diperuntukkan untuk kavling lebih dari 25.000 kaki persegi 
- Perbedaan rasial dalam kavling
- Jarak tertimbang ke lima pusat pekerjaan di Boston.  
  
Harga rumah di Boston paling dipengaruhi oleh rata-rata jumlah kamar, karena variabel ini memiliki korelasi tertinggi terhadap harga. Semakin banyak jumlah kamar, semakin besar pula daya tarik rumah tersebut bagi calon pembeli, karena mereka cenderung mencari ruang yang lebih luas dan nyaman untuk ditinggali bersama keluarga atau untuk investasi jangka panjang.

#### 2. Tingkat kejahatan berpengaruh negatif terhadap harga rumah

In [ ]:
sns.scatterplot(x='CRIM', y='PRICE', data=data)
plt.title('Hubungan antara Crime Rate dan Harga Rumah')
plt.xlabel('Tingkat Kejahatan (CRIM)')
plt.ylabel('Harga Rumah (PRICE)')
plt.show()

**hasil :**  
Dari grafik scatterplot terlihat pola yang cukup jelas: semakin tinggi tingkat kejahatan di suatu area, semakin rendah harga rumahnya. Misalnya, pada area dengan tingkat kejahatan antara 0 hingga 20, harga rumah cenderung tinggi dan stabil, ini bisa disebut sebagai zona "aman" yang paling diminati. Sebaliknya, pada area dengan tingkat kejahatan di atas 60, harga rumah anjlok tajam, menunjukkan bahwa keamanan menjadi faktor krusial dalam menentukan nilai properti.

#### 3. Rasio murid-guru berkorelasi dengan harga rumah?

In [ ]:
corr = data[['PTRATIO', 'PRICE']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(data[['PTRATIO', 'PRICE']].corr(), annot=True, cmap='coolwarm')
plt.title("Korelasi antara PTRATIO dan PRICE")
plt.show()

**hasil :**  
Korelasi sebesar -0.51 antara rasio murid-guru (PTRATIO) dan harga rumah (PRICE) menunjukkan bahwa ada hubungan negatif sedang antara kedua variabel tersebut. Artinya:  
- Semakin tinggi rasio murid-guru (lebih banyak murid per guru, kualitas pendidikan cenderung menurun)
- Semakin rendah harga rumah.  
  
Masyarakat cenderung menilai kualitas pendidikan sebagai faktor penting dalam memilih tempat tinggal. Jika suatu daerah memiliki rasio murid-guru yang tinggi, itu bisa menjadi indikator bahwa sekolah di sana terlalu padat atau kurang tenaga pengajar, yang membuat wilayah tersebut kurang diminati, sehingga menurunkan nilai properti.

#### 4. Kedekatan dengan Sungai berdampak signifikan terhadap harga rumah

In [ ]:
data.groupby('CHAS')['PRICE'].mean()

In [ ]:
sns.boxplot(x='CHAS', y='PRICE', data=data)
plt.xticks([0, 1], ['Tidak Dekat Sungai', 'Dekat Sungai'])
plt.title("Perbandingan Harga Rumah Berdasarkan Kedekatan dengan Sungai")
plt.show()

**hasil :**  
Secara umum rata - rata harga rumah yang berada di dekat sungai cenderung lebih mahal, Kemudian berdasarkan bloxplot:  
- Median harga rumah untuk CHAS = 1 lebih tinggi dibanding CHAS = 0 → menunjukkan bahwa rumah yang dekat dengan sungai cenderung memiliki harga lebih mahal.
- Boxplot CHAS = 1 memiliki kotak yang lebih lebar → artinya harga rumah yang dekat sungai memiliki variasi yang lebih besar, atau rentang harga yang lebih lebar.
- Terdapat outlier pada CHAS = 0 yang berada di atas nilai maksimum → menandakan bahwa meskipun rumah yang jauh dari sungai cenderung lebih murah, ada beberapa rumah yang berharga sangat tinggi, namun jarang terjadi.  
  
Rumah yang berada dekat dengan sungai (CHAS = 1) cenderung memiliki harga lebih tinggi dan lebih bervariasi dibanding rumah yang jauh dari sungai. Kehadiran outlier pada CHAS = 0 menunjukkan ada pengecualian di mana beberapa rumah jauh dari sungai bisa memiliki harga tinggi, meskipun jarang terjadi.

#### 5. Rumah di area dengan pajak tinggi memiliki harga jual lebih rendah

In [ ]:
correlation = data['TAX'].corr(data['PRICE'])
print(f'Korelasi antara TAX dan PRICE: {correlation}')

In [ ]:
data['TAX_Group'] = pd.qcut(data['TAX'], q=3, labels=['Rendah', 'Sedang', 'Tinggi'])
print(data.groupby('TAX_Group')['PRICE'].mean())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x='TAX_Group', y='PRICE', data=data)
plt.title("Perbandingan Harga Rumah Berdasarkan Kelompok Pajak")
plt.show()

**hasil :**  
Dari hasil korelasi antara tinggi pajak terhadap harga rumah adalah -0.47 yang berarti semakin tinggi pajak maka harga rumah semakin turun, kemudian dengan membagi tinggi pajak menjadi tiga bagian yaitu rendah, sedang, dan tinggi mendapatkan hasil:  
- Median harga rumah pada kelompok pajak tinggi lebih rendah dibanding kelompok pajak sedang dan rendah. Hal ini menunjukkan bahwa beban pajak yang tinggi kemungkinan besar menurunkan daya tarik sebuah properti, sehingga menekan harga jualnya.
- Boxplot kelompok pajak tinggi memiliki kotak yang sempit, yang berarti sebaran harga rumah di area dengan pajak tinggi lebih homogen dan cenderung terkonsentrasi pada kisaran harga yang lebih rendah.
- Terdapat banyak outlier pada kelompok pajak sedang, sementara kelompok pajak tinggi hanya memiliki sedikit outlier. Ini mengindikasikan bahwa di wilayah pajak sedang, variasi harga rumah jauh lebih besar dan ada rumah-rumah yang bernilai sangat tinggi, sedangkan di wilayah pajak tinggi, harga rumah cenderung stabil namun tetap rendah.  
  
Hasil analisis mendukung research question bahwa rumah di area dengan pajak tinggi memiliki harga jual yang lebih rendah. Hal ini ditunjukkan oleh korelasi negatif sebesar -0.47 antara pajak dan harga rumah, serta median harga yang lebih rendah pada kelompok pajak tinggi. Sebaran harga yang sempit di area pajak tinggi juga menunjukkan bahwa harga rumah cenderung stabil namun tetap rendah di wilayah tersebut.

### Linear Regression

In [ ]:
data =pd.read_csv('data/housing.csv', sep='\s+', header=None)
data.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']

In [ ]:
X = data.drop('PRICE', axis=1)
y = data['PRICE']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
model.coef_
model.intercept_

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

In [ ]:
result_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

plt.figure(figsize=(8, 5))
sns.scatterplot(x='Actual', y='Predicted', data=result_df)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Ideal (y = x)')

plt.title('Actual vs Predicted Harga Rumah (Test Set)')
plt.xlabel('Harga Rumah Aktual')
plt.ylabel('Harga Rumah Prediksi')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#### Normalizer Scalling Before Remove Outliers

In [ ]:
X = data.drop('PRICE', axis=1)
y = data['PRICE']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
normalizer = Normalizer()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

#### Remove Outliers

In [ ]:
# Check for outliers using boxplots:
numeric_cols = data.select_dtypes(exclude='object').columns
n_cols = 4
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
plt.figure(figsize=(5 * n_cols, 4 * n_rows))
for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(y=data[col])
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
# Use IQR method to identify outliers for numerical features:
for col in data.select_dtypes(include=np.number).columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    print(f"Outliers in {col}: {outliers.shape[0]}")

In [ ]:
data.shape

In [ ]:
numeric_cols = data.select_dtypes(include='number').columns
for col in numeric_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    data[col] = data[col].clip(lower=lower, upper=upper)

In [ ]:
plt.figure(figsize=(15, 10))
for i, col in enumerate(data.columns):
    plt.subplot(4, 4, i + 1)
    sns.boxplot(y=data[col])
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
# Use IQR method to identify outliers for numerical features:
for col in data.select_dtypes(include=np.number).columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    print(f"Outliers in {col}: {outliers.shape[0]}")

In [ ]:
data.shape

#### Normalizer Scaling After Remove Ouliers

In [ ]:
X = data.drop('PRICE', axis=1)
y = data['PRICE']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
normalizer = Normalizer()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

In [ ]:
result_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

plt.figure(figsize=(8, 5))
sns.scatterplot(x='Actual', y='Predicted', data=result_df)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Ideal (y = x)')

plt.title('Actual vs Predicted Harga Rumah (Test Set)')
plt.xlabel('Harga Rumah Aktual')
plt.ylabel('Harga Rumah Prediksi')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
for i in range(5):
    print(f"predicated: {y_pred[i]:.2f} | Actual: {y[i]}")

#### Conclusion Linear Regression

Berdasarkan hasil dari normalizer scaling sebelum penghapusan outlier, model regresi linear menghasilkan:

* Mean Absolute Error (MAE): 3.06
* Mean Squared Error (MSE): 23.11
* R² Score: 0.68

Setelah dilakukan penghapusan outlier, performa model mengalami peningkatan signifikan dengan hasil:

* MAE turun menjadi: 2.39
* MSE turun menjadi: 12.44
* R² meningkat menjadi: 0.75

**Jadi**, penghapusan outlier terbukti mampu meningkatkan akurasi dan kestabilan model, di mana error menjadi lebih kecil dan model mampu menjelaskan variasi data lebih baik. Ini menunjukkan bahwa outlier memiliki dampak negatif terhadap performa model prediksi harga rumah, sehingga penting untuk melakukan pembersihan data sebelum membangun model.

---

Untuk grafik regresi linear setelah menggunakan normalizer scaling, terlihat bahwa **sebelum penghapusan outlier**, titik-titik data pada plot cenderung tersebar jauh dari garis regresi, menandakan bahwa prediksi model tidak konsisten.

Namun, **setelah dilakukan penghapusan outlier**, distribusi titik menjadi lebih rapat di sekitar garis regresi, menunjukkan bahwa model semakin akurat dan stabil dalam memprediksi harga rumah. Hal ini memperkuat temuan sebelumnya bahwa outlier dapat mengganggu pola data dan menurunkan performa model, baik secara visual maupun secara metrik evaluasi.

